# Linebase model

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("redoute_v3.csv", index_col = 0).reset_index().drop("id", axis = 1)
df.head()

,rating,title,comment,createdAt,transaction,SupplierReply,SupplierReplyDate,date_of_cmt,year_of_cmt,month_of_cmt,...,year_of_transaction,month_of_transaction,day_of_transaction,weekday_of_transaction,transaction_in_weekend,transaction_date,comment_length,log_comment_length,title_length,nb_days_before_review
0,4,Facilte,"Facilte, sollicitations dosées...qualité..on p...",2023-10-05 14:18:42,2023-09-23,NaN,NaN,2023-10-05,2023,10,...,2023,9,23,5,1,2023-09-23,1049,6.955593,7,12.0
1,4,Jolie table mais plus pour 6 que pour 8,Livraison impeccable avec possibilité de prend...,2023-10-02 13:08:27,2023-09-11,NaN,NaN,2023-10-02,2023,10,...,2023,9,11,0,0,2023-09-11,885,6.785588,39,21.0
2,4,Achat d'un canapé La Redoute intérieurs…,Achat d'un canapé La Redoute intérieurs le 6/9...,2023-09-18 16:52:21,2023-09-06,NaN,NaN,2023-09-18,2023,9,...,2023,9,6,2,0,2023-09-06,713,6.569481,40,12.0
3,4,excellent site mais des changements qui compl...,je commande depuis longtemps chez la redoute; ...,2023-10-13 08:57:16,2023-10-02,NaN,NaN,2023-10-13,2023,10,...,2023,10,2,0,0,2023-10-02,712,6.568078,66,11.0
4,5,"Comme d'habitude, aucun souci et rapidité d'ex...","Comme d'habitude, aucun souci et rapidité d'ex...",2023-09-21 11:09:15,2023-09-15,NaN,NaN,2023-09-21,2023,9,...,2023,9,15,4,0,2023-09-15,639,6.459904,200,6.0


In [3]:
# Prepare the data for a regression model to predicts the rating with comment_lenght, title_length and nb_days_before_review
data = df[["comment_length", "title_length", "nb_days_before_review"]]
#data.head()
target = df["rating"]
target.head()

#for sentiment analysis .Reduce the target to 3 value -1, 0 and 1
sentiment = 0*(target <= 3)  + 1*(target >= 4)
print(sentiment.value_counts())
print(target.value_counts())

rating
1    20769
0     2700
Name: count, dtype: int64
rating
5    17026
4     3743
3     1228
1      760
2      712
Name: count, dtype: int64


In [4]:
# Split data into train data and test data with test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state = 123)

In [5]:
# Logistic Regression Model

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

# Define a pipeline with 

scaler = StandardScaler()
lreg = LogisticRegression()

lreg_pipe = Pipeline(steps = [("scaling", scaler), ("logistic_reg", lreg)])
#params = {"logistic_reg__penalty": ["l1", "l2", "elasticnet", None], "logistic_reg__class_weight": ["balanced"]}
params = {"logistic_reg__penalty": ["l2"], 
          "logistic_reg__class_weight": ["balanced"],
          "logistic_reg__C": np.logspace(-2, 2, 5)}

print("Start GridSearch ...")
t0 = time()

clf = GridSearchCV(estimator = lreg_pipe, param_grid = params, cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future 

Start GridSearch ...


/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future 

Done in 0.465s
{'logistic_reg__C': 0.01, 'logistic_reg__class_weight': 'balanced', 'logistic_reg__penalty': 'l2'}
score on train data: 0.31954459203036056
Score on tests data: 0.5944103612815269
              precision    recall  f1-score   support

           1       0.10      0.30      0.15       196
           2       0.14      0.26      0.18       185
           3       0.13      0.19      0.15       281
           4       0.19      0.13      0.15       926
           5       0.82      0.75      0.78      4280

    accuracy                           0.59      5868
   macro avg       0.28      0.33      0.28      5868
weighted avg       0.64      0.59      0.61      5868



/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_spar

Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,58,42,24,27,45
2,49,49,27,17,43
3,65,57,53,35,71
4,96,83,69,116,562
5,308,120,232,408,3212


In [6]:
# Logistic Regression Model

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

scaler = StandardScaler()
lreg = LogisticRegression()

print("Start GridSearch ...")
t0 = time()

lreg_pipe = Pipeline(steps = [("scaling", scaler), ("logistic_reg", lreg)])
#params = {"logistic_reg__penalty": ["l1", "l2", "elasticnet", None], "logistic_reg__class_weight": ["balanced"]}
params = {"logistic_reg__solver": ["lbfgs"],
          "logistic_reg__penalty": ["l2"], 
         # "logistic_reg__class_weight": ["balanced"],
          "logistic_reg__C": [1e-7, 6e-7, 8e-7, 1e-6, 2e-6]
          }
#"logistic_reg__max_iter": [1500]
#"logistic_reg__C": [0.001, 0.002, 0.01, 0.1, 0.5, 1, 10, 50] ==> score test = 0.6

clf = GridSearchCV(estimator = lreg_pipe, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future 

Start GridSearch ...
Done in 0.327s
{'logistic_reg__C': 1e-07, 'logistic_reg__penalty': 'l2', 'logistic_reg__solver': 'lbfgs'}
score on train data: 0.1870198500145735
Score on tests data: 0.6087731576479081
              precision    recall  f1-score   support

           1       0.10      0.31      0.15       196
           2       0.12      0.36      0.18       185
           3       0.00      0.00      0.00       281
           4       0.10      0.01      0.02       926
           5       0.79      0.84      0.82      4280

    accuracy                           0.64      5868
   macro avg       0.22      0.31      0.23      5868
weighted avg       0.60      0.64      0.61      5868



/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_spar

Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,61,55,0,5,75
2,50,67,0,1,67
3,67,83,0,3,128
4,104,118,0,10,694
5,341,237,3,86,3613


In [7]:
1e-7+0.5*1e-6

6e-07

Start GridSearch ...
Done in 1.072s
{'logistic_reg__C': 0.0001, 'logistic_reg__class_weight': 'balanced', 'logistic_reg__max_iter': 1500, 'logistic_reg__penalty': 'l2', 'logistic_reg__solver': 'lbfgs'}
score on train data: 0.6382591948310139
Score on tests data: 0.639229720518064

<b>Best performance</b>
- {'logistic_reg__C': 1e-06, 'logistic_reg__class_weight': 'balanced', 'logistic_reg__penalty': 'l2', 'logistic_reg__solver': 'lbfgs'}
- score on train data: 0.6591101710836106
- Score on tests data: 0.662747102931152

In [8]:
# Support Vector Machine model 1

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

scaler = StandardScaler()
svc = SVC()

svc_pipe = Pipeline(steps = [("scaling", scaler), ("svc", svc)])
params = {"svc__kernel": ['linear']}
#params = {"svc__kernel": ['linear', 'rbf', 'sigmoid']}

print("Start GridSearch ...")
t0 = time()

clf = GridSearchCV(estimator = svc_pipe, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

Start GridSearch ...
Done in 1.436s
{'svc__kernel': 'linear'}
score on train data: 0.26658698625222027
Score on tests data: 0.6135641732529715
              precision    recall  f1-score   support

           1       0.10      0.28      0.15       196
           2       0.15      0.10      0.12       185
           3       0.12      0.38      0.18       281
           4       0.26      0.04      0.07       926
           5       0.81      0.79      0.80      4280

    accuracy                           0.61      5868
   macro avg       0.29      0.32      0.26      5868
weighted avg       0.65      0.61      0.61      5868



Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,54,20,58,8,56
2,45,18,74,4,44
3,52,28,107,14,80
4,86,25,175,40,600
5,297,29,485,86,3383


In [9]:
# Support Vector Machine model 2

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

scaler = StandardScaler()
svc = SVC()

svc_pipe = Pipeline(steps = [("scaling", scaler), ("svc", svc)])

params = {"svc__kernel": ['linear'],
          "svc__C":[0.01, 0.5, 1, 10],
          "svc__gamma": ['scale', 'auto', 0.001, 0.02, 0.05, 0.5]
         }


print("Strating GridSearch ...")
t0 = time()

clf = GridSearchCV(estimator = svc_pipe, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

Strating GridSearch ...
Done in 31.649s
{'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}
score on train data: 0.2706516302718255
Score on tests data: 0.6117008506222067
              precision    recall  f1-score   support

           1       0.11      0.26      0.15       196
           2       0.15      0.10      0.12       185
           3       0.12      0.35      0.18       281
           4       0.16      0.05      0.07       926
           5       0.81      0.79      0.80      4280

    accuracy                           0.61      5868
   macro avg       0.27      0.31      0.27      5868
weighted avg       0.63      0.61      0.61      5868



Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,50,23,56,13,54
2,43,19,63,6,54
3,52,31,99,9,90
4,75,26,163,44,618
5,256,26,411,200,3387


In [10]:
# KNN Classifier

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

params = {"n_neighbors": [20, 30, 31, 32,33, 35],
          "weights":["uniform", "distance"],
          "p": [1, 2],
          "n_jobs": [-1]
         }

print("Strating GridSearch ...")
t0 = time()

clf = GridSearchCV(estimator = knn, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

Strating GridSearch ...
Done in 5.507s
{'n_jobs': -1, 'n_neighbors': 35, 'p': 2, 'weights': 'uniform'}
score on train data: 0.30286465812008556
Score on tests data: 0.48042516367674515
              precision    recall  f1-score   support

           1       0.10      0.31      0.15       196
           2       0.08      0.29      0.13       185
           3       0.09      0.31      0.14       281
           4       0.17      0.24      0.20       926
           5       0.84      0.46      0.59      4280

    accuracy                           0.41      5868
   macro avg       0.26      0.32      0.24      5868
weighted avg       0.65      0.41      0.48      5868



Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,61,53,40,24,18
2,48,53,42,18,24
3,49,82,87,28,35
4,115,117,170,220,304
5,349,339,580,1040,1972


In [11]:
# Gradient Boosting Classifier

# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators = 100,
                                 learning_rate = 1.0,
                                 max_depth = 1,
                                 random_state = 0)

clf.fit(X_ru, y_ru),


print("score on train data:", clf.score(X_train, y_train))
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

score on train data: 0.4691778876200216
Score on tests data: 0.47119972733469667
              precision    recall  f1-score   support

           1       0.09      0.22      0.13       196
           2       0.07      0.27      0.11       185
           3       0.11      0.28      0.16       281
           4       0.18      0.21      0.19       926
           5       0.83      0.56      0.67      4280

    accuracy                           0.47      5868
   macro avg       0.26      0.31      0.25      5868
weighted avg       0.64      0.47      0.53      5868



Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,44,42,53,20,37
2,40,50,50,12,33
3,45,77,78,41,40
4,83,138,129,192,384
5,254,424,392,809,2401


# Adjust the target to :(i) -1 if rating = 1, 2 or 3, (ii) 1 if rating = 4 or 5

In [12]:
# Split data into train data and test data with test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(data, sentiment, test_size=0.25, random_state = 123)

In [13]:
# Logistic Regression Model
# Define a pipeline with 
# Undersampling
ru = RandomUnderSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

scaler = StandardScaler()
lreg = LogisticRegression()

lreg_pipe = Pipeline(steps = [("scaling", scaler), ("logistic_reg", lreg)])
#params = {"logistic_reg__penalty": ["l1", "l2", "elasticnet", None], "logistic_reg__class_weight": ["balanced"]}
params = {"logistic_reg__penalty": ["l2"], 
          #"logistic_reg__class_weight": ["balanced"],
          "logistic_reg__C": np.logspace(-2, 2, 5)}

clf = GridSearchCV(estimator = lreg_pipe, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)
print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

{'logistic_reg__C': 1.0, 'logistic_reg__penalty': 'l2'}
score on train data: 0.7413642243025664
Score on tests data: 0.8192309162112408
              precision    recall  f1-score   support

          -1       0.30      0.69      0.42       662
           1       0.95      0.80      0.87      5206

    accuracy                           0.79      5868
   macro avg       0.63      0.74      0.65      5868
weighted avg       0.88      0.79      0.82      5868



Classe prédite,-1,1
Classe réelle,,
-1,457,205
1,1043,4163


In [14]:
scaler = StandardScaler()
lreg = LogisticRegression()

print("Start GridSearch ...")
t0 = time()

lreg_pipe = Pipeline(steps = [("scaling", scaler), ("logistic_reg", lreg)])
#params = {"logistic_reg__penalty": ["l1", "l2", "elasticnet", None], "logistic_reg__class_weight": ["balanced"]}
params = {"logistic_reg__solver": ["lbfgs", "sag"],
          "logistic_reg__penalty": ["l2"], 
          "logistic_reg__class_weight": ["balanced"],
          "logistic_reg__C": [0.001, 0.002, 0.01, 0.1, 0.5, 1, 10, 50],
          "logistic_reg__max_iter": [1500]}

clf = GridSearchCV(estimator = lreg_pipe, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_train, y_train)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

Start GridSearch ...
Done in 2.272s
{'logistic_reg__C': 0.001, 'logistic_reg__class_weight': 'balanced', 'logistic_reg__max_iter': 1500, 'logistic_reg__penalty': 'l2', 'logistic_reg__solver': 'lbfgs'}
score on train data: 0.8195348820873651
Score on tests data: 0.8273846548824573
              precision    recall  f1-score   support

          -1       0.32      0.67      0.43       662
           1       0.95      0.82      0.88      5206

    accuracy                           0.80      5868
   macro avg       0.63      0.74      0.65      5868
weighted avg       0.88      0.80      0.83      5868



Classe prédite,-1,1
Classe réelle,,
-1,445,217
1,963,4243


In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

#ru = RandomUnderSampler()
ru = RandomOverSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

params = {"n_neighbors": [4, 5, 6, 7, 8, 9],
          "weights":["uniform", "distance"],
          "p": [1, 2],
          "n_jobs": [-1]
         }

print("Strating GridSearch ...")
t0 = time()

clf = GridSearchCV(estimator = knn, param_grid = params, scoring = "f1_weighted", cv = 5)
clf.fit(X_ru, y_ru)

print("Done in %0.3fs" % (time() - t0))

print(clf.best_params_)
print("score on train data:", clf.best_score_)
print("Score on tests data:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

Strating GridSearch ...
Done in 21.339s
{'n_jobs': -1, 'n_neighbors': 4, 'p': 2, 'weights': 'distance'}
score on train data: 0.883755718525433
Score on tests data: 0.8126710615637313
              precision    recall  f1-score   support

          -1       0.25      0.44      0.32       662
           1       0.92      0.83      0.88      5206

    accuracy                           0.79      5868
   macro avg       0.59      0.63      0.60      5868
weighted avg       0.85      0.79      0.81      5868



Classe prédite,-1,1
Classe réelle,,
-1,288,374
1,861,4345


In [16]:
# Gradient Boosting Classifier
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

ru = RandomOverSampler()
(X_ru, y_ru) = ru.fit_resample(X_train, y_train)

clf = GradientBoostingClassifier(n_estimators = 100,
                                 learning_rate = 1.0,
                                 max_depth = 1,
                                 random_state = 0)


clf.fit(X_ru, y_ru),
y_pred = clf.predict(X_test)

print("score on train data:", clf.score(X_train, y_train))
print("Score on tests data:", clf.score(X_test, y_test))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames=["Classe prédite"])

score on train data: 0.763649792625419
Score on tests data: 0.7631220177232447
              precision    recall  f1-score   support

          -1       0.28      0.70      0.40       662
           1       0.95      0.77      0.85      5206

    accuracy                           0.76      5868
   macro avg       0.62      0.74      0.63      5868
weighted avg       0.88      0.76      0.80      5868



Classe prédite,-1,1
Classe réelle,,
-1,463,199
1,1191,4015


In [ ]:
## Neural network model